## Searching for IRS1 sequence

In [1]:
from Bio import Entrez, SeqIO, Medline
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re

In [16]:
#pubmed search
database = 'PubMed'
word = 'irs1 insulin'
res= 15 
email= "karynalysenko@ua.pt"

Entrez.email= email
handle=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle)
handle.close()
idlist= record['IdList']

handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text")
records = list(Medline.parse(handle))


for record in records[0:5]:
    #print(record) 
    #print("PMID:",record.get("PMID","-"))
    print("Title:", record.get("TI", "-"))
    #print('abstract:', record.get('AB', '-'))
    print("Authors:", record.get("AU", "-"))
    print("Source:", record.get("SO", "-"))
    print("")

Title: Effect of Codonopsis Radix and Polygonati Rhizoma on the regulation of the IRS1/PI3K/AKT signaling pathway in type 2 diabetic mice.
Authors: ['Mao YP', 'Song YM', 'Pan SW', 'Li N', 'Wang WX', 'Feng BB', 'Zhang JH']
Source: Front Endocrinol (Lausanne). 2022 Dec 14;13:1068555. doi: 10.3389/fendo.2022.1068555. eCollection 2022.

Title: The identities of insulin signaling pathway are affected by overexpression of Tau and its phosphorylation form.
Authors: ['Ma N', 'Liang Y', 'Yue L', 'Liu P', 'Xu Y', 'Zhu C']
Source: Front Aging Neurosci. 2022 Dec 16;14:1057281. doi: 10.3389/fnagi.2022.1057281. eCollection 2022.

Title: Insulin ameliorates dim blue light at night-induced apoptosis in hippocampal neurons via the IR/IRS1/AKT/GSK3beta/beta-catenin signaling pathway.
Authors: ['Liu Q', 'Wang Z', 'Cao J', 'Dong Y', 'Chen Y']
Source: Ecotoxicol Environ Saf. 2022 Dec 29;250:114488. doi: 10.1016/j.ecoenv.2022.114488.

Title: Sodium oxamate reduces lactate production to improve the glucose h

In [23]:
#ncbi search to find the sequence of IRS1
database = 'nucleotide'
word = 'irs1 and homo sapiens and Chromosome 2 and not predicted and not unverified '
res= '15'
email= 'karyanlysenko@ua.pt'
Entrez.email= email
handle_search=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle_search)
handle_search.close()
idlist= record['IdList']
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    print(info.id, '-', info.description)
    #print('length of seq:', len(info.seq)) #to check the length of the sequences

NM_005544.3 - Homo sapiens insulin receptor substrate 1 (IRS1), mRNA
NM_001100818.2 - Homo sapiens phosphotyrosine interaction domain containing 1 (PID1), transcript variant 2, mRNA
NM_001330158.2 - Homo sapiens phosphotyrosine interaction domain containing 1 (PID1), transcript variant 5, mRNA
NM_001330157.2 - Homo sapiens phosphotyrosine interaction domain containing 1 (PID1), transcript variant 4, mRNA
NM_017933.5 - Homo sapiens phosphotyrosine interaction domain containing 1 (PID1), transcript variant 1, mRNA
NM_001330156.1 - Homo sapiens phosphotyrosine interaction domain containing 1 (PID1), transcript variant 3, mRNA
NG_015830.1 - Homo sapiens insulin receptor substrate 1 (IRS1), RefSeqGene on chromosome 2
CM000253.1 - Homo sapiens chromosome 2, whole genome shotgun sequence
CH471063.1 - Homo sapiens 211000035834619 genomic scaffold, whole genome shotgun sequence


The selection of the id has to be done manually as there is no pattern in writing the titles of the queries.\
The id __NG_015830.1__ is the only one where the annotated sequence is not a whole genome of the chromossome, is not a mRNA and actually is RefSeq. This means that the sequence is being used as a standard for well-characterized genes. So id __NG_015830.1__ will be used from now on.

In [7]:
#The correspondent information of NG_015830.1 was downloaded to a file
import os
Entrez.email = "karynalysenko@ua.pt"
filename = "NG_015830_1.gb"
if not os.path.isfile(filename):
    net_handle = Entrez.efetch(db="nucleotide", id="NG_015830.1", rettype="gb", retmode="text")
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()

In [25]:
record = SeqIO.read(open("NG_015830_1.gb"), format="genbank")
position=0
record_types={}
for x in record.features:
    record_types[x.type]=record_types.get(x.type,0)+1
for i in range(len(record.features)):
    if record.features[i].type == "CDS":
        position=i
print("The length of the sequence: {}\n".format(len(record.seq)))
print("Type of features: {}\n".format(record_types))
print("Comment from NCBI: {}\n".format(record.annotations["comment"]))
#checking the location of the CDS on the original sequence
print("Location of the CDS on the original sequence: {}".format(record.features[position].location))

The length of the sequence: 74474

Type of features: {'source': 1, 'gene': 2, 'mRNA': 2, 'exon': 2, 'CDS': 1, 'misc_feature': 46}

Comment from NCBI: REVIEWED REFSEQ: This record has been curated by NCBI staff. The
reference sequence was derived from AC010735.11.
This sequence is a reference standard in the RefSeqGene project.
Summary: This gene encodes a protein which is phosphorylated by
insulin receptor tyrosine kinase. Mutations in this gene are
associated with type II diabetes and susceptibility to insulin
resistance. [provided by RefSeq, Nov 2009].

Location of the CDS on the original sequence: [5052:8781](+)


In [34]:
CDS_nuc_seq_location=record.seq[int(record.features[position].location.start):int(record.features[position].location.end)]

In [33]:
#saving the CDS_nucleotides and CD_aminoacid seqs in files
filename = "NG_015830_1.gb"
filename_CDS_nucl = "CDS_nucleot_seq-test.fasta"
filename_CDS_prot = "CDS_prot_seq.fasta"
input_handle  = open(filename, "r")
output_handle_nucl = open(filename_CDS_nucl, "w")
output_handle_prot = open(filename_CDS_prot, "w")
for seq_record in SeqIO.parse(input_handle, "genbank") :
    output_handle_nucl.write(">\n%s" % (CDS_nuc_seq_location))
    output_handle_prot.write(">\n%s" % ("".join(record.features[position].qualifiers['translation']))) #without join, output is a list
    
output_handle_nucl.close()
output_handle_prot.close()
input_handle.close()

## BLASTN - for all organisms

In [4]:
record_blastn = SeqIO.read(open("CDS_nucleot_seq.fasta"), format="fasta") 
print(len(record_blastn.seq))

3729


In [41]:
Blastn = NCBIWWW.qblast("blastn", "nt", record_blastn.seq) #not filtered for Homo sapiens
with open('blastn_CDS_nucleot_seq.xml', "w") as out_handle:
        out_handle.write(Blastn.read())
Blastn.close()

In [46]:
results_Blastn = open("blastn_CDS_nucleot_seq.xml")
blastn_records = NCBIXML.read(results_Blastn)
for parameter in blastn_records.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)

Number of alignments: 50

Accession:  XM_047444224
Definition:  PREDICTED: Homo sapiens insulin receptor substrate 1 (IRS1), transcript variant X2, mRNA
E-value:  0.0

Accession:  XM_047444223
Definition:  PREDICTED: Homo sapiens insulin receptor substrate 1 (IRS1), transcript variant X1, mRNA
E-value:  0.0

Accession:  NM_005544
Definition:  Homo sapiens insulin receptor substrate 1 (IRS1), mRNA
E-value:  0.0

Accession:  NG_015830
Definition:  Homo sapiens insulin receptor substrate 1 (IRS1), RefSeqGene on chromosome 2
E-value:  0.0

Accession:  AC010735
Definition:  Homo sapiens BAC clone RP11-395N3 from 2, complete sequence
E-value:  0.0


In [49]:
#filtering the "predicted" alignments
results_Blastn= open('blastn_CDS_nucleot_seq.xml')
blastn_records = NCBIXML.read(results_Blastn)
E_VALUE_THRESH = 0.001
count_preditc, count_homo=0,0
list_filtered_alignments=[]
for alignment in  blastn_records.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            exist = re.search(r'PREDICTED:\s', alignment.title)
            if exist:
                # print( existe[0] )
                pre = re.match(r'PREDICTED:\s', exist[0] )
                if pre:
                    count_preditc+=1
                    #print(id)
            else:
                list_filtered_alignments.append(alignment.accession)
                homo=re.search(r'Homo\ssapiens',alignment.title)
                if homo:
                    count_homo+=1
print(list_filtered_alignments)
print('Total {} PREDICTED seqs found and remaining {} ids are from Homo sapiens'.format(count_preditc, count_homo))
#counting of Homo sapiens it's not conclusive, just to have an ideia, because the titles don't follow any pattern

['NM_005544', 'NG_015830', 'AC010735', 'S62539', 'BC053895', 'LT743046', 'KJ891488', 'AB384351', 'EU831611', 'EU831698', 'S85963', 'U43502']
Total 38 PREDICTED seqs found and remaining 8 ids are from Homo sapiens


In [ ]:
with open('CDS_nucleotide_result_blast.txt', 'w') as f:
    for line in list_filtered_alignments:
        f.write(f"{line}\n")

## BLASTP

In [14]:
record_blastp = SeqIO.read(open("CDS_prot_seq.fasta"), format="fasta") 
print(len(record_blastp.seq))

1242


In [15]:
Blastp = NCBIWWW.qblast("blastp", "swissprot", record_blastp.seq) #not filtered for Homo sapiens
with open('blastp_CDS_prot_seq.xml', "w") as out_handle:
        out_handle.write(Blastp.read())
Blastp.close()

In [61]:
results_Blastp = open("blastp_CDS_prot_seq.xml")
blastp_records = NCBIXML.read(results_Blastp)
for parameter in blastp_records.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)


Accession:  P35568
Definition:  RecName: Full=Insulin receptor substrate 1; Short=IRS-1 [Homo sapiens]
E-value:  0.0

Accession:  Q28224
Definition:  RecName: Full=Insulin receptor substrate 1; Short=IRS-1 [Chlorocebus aethiops]
E-value:  0.0

Accession:  P35570
Definition:  RecName: Full=Insulin receptor substrate 1; Short=IRS-1; AltName: Full=pp185 [Rattus norvegicus]
E-value:  0.0

Accession:  P35569
Definition:  RecName: Full=Insulin receptor substrate 1; Short=IRS-1 [Mus musculus]
E-value:  0.0

Accession:  P84770
Definition:  RecName: Full=Insulin receptor substrate 1-B; Short=IRS1-B; Short=xIRS-1-B; AltName: Full=XIRS-L' [Xenopus laevis]
E-value:  0.0


In [60]:
results_Blastp= open('blastp_CDS_prot_seq.xml')
blastp_records = NCBIXML.read(results_Blastp)
E_VALUE_THRESH = 0.001
list_filtered_alignments,list_species=[],[]
for alignment in  blastp_records.alignments:
    for hsp in alignment.hsps:
#         print(hsp.identities)    # maybe add more 
        if hsp.expect < E_VALUE_THRESH:
            list_filtered_alignments.append(alignment.accession)
            title_organism=re.search(r'\[.+\s.+\]', alignment.title)
            if title_organism:
                m = re.match(r'\[.+\s.+\]', title_organism[0] )
                specie = m.group(0)
                #print(specie)
                list_species.append(specie)
for x in sorted(set(list_species)):
    print("number of times: {} that appeared specie: {}".format(list_species.count(x),x))

print(list_filtered_alignments)

number of times: 1 that appeared specie: [Bos taurus]
number of times: 1 that appeared specie: [Chlorocebus aethiops]
number of times: 1 that appeared specie: [Drosophila ananassae]
number of times: 1 that appeared specie: [Drosophila erecta]
number of times: 1 that appeared specie: [Drosophila melanogaster]
number of times: 1 that appeared specie: [Drosophila sechellia]
number of times: 1 that appeared specie: [Drosophila yakuba]
number of times: 9 that appeared specie: [Homo sapiens]
number of times: 1 that appeared specie: [Mesocricetus auratus]
number of times: 6 that appeared specie: [Mus musculus]
number of times: 2 that appeared specie: [Rattus norvegicus]
number of times: 4 that appeared specie: [Xenopus laevis]
number of times: 2 that appeared specie: [Xenopus tropicalis]
['P35568', 'Q28224', 'P35570', 'P35569', 'P84770', 'Q91615', 'Q9DF49', 'Q5RJW5', 'P81122', 'Q9Y4H2', 'Q9Y4H2', 'Q6P4Y6', 'Q6P4Y6', 'O14654', 'Q9Z0Y7', 'B3N946', 'B4NZ70', 'B4HWI2', 'Q9XTN2', 'B3MPN6', 'Q9Z1S8

The output of Blastp gave more hits. So the list of ids of the last script will be saved and used on Uniprot search.

In [57]:
with open('CDS_protein_result_blastp.txt', 'w') as f:
    for line in list_filtered_alignments:
        f.write(f"{line}\n")

## Uniprot search of Blastp results

In [ ]:
#the seqs here are already aligned, they have '-' (from blastp)
# results_Blastp= open('blastp_CDS_prot_seq.xml')
# blastp_records = NCBIXML.read(results_Blastp)

# with open('allOrg_CDS_prot.fasta', 'w') as f:
#     for alignment in  blastp_records.alignments:
#         f.write(f">{alignment.title}\n")
#         for hsp in alignment.hsps:
#             f.write(f"{hsp.query}\n\n")

In [202]:
#saving sequences from Uniprot
results_Blastp= open('blastp_CDS_prot_seq.xml')
blastp_records = NCBIXML.read(results_Blastp)
titles_list=[]
for alignment in  blastp_records.alignments:   
    titles_list.append(alignment.title)

file= open("CDS_protein_result_blastp.txt", "r")
fields="sequence"
WEBSITE_API="https://rest.uniprot.org"
seqs=[]

for i in file:
    r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i, fields))
    seqs.append(str(r.content))

with open('allOrg_CDS_prot_fromUniprot.fasta', 'w') as f:
    for index, seq in enumerate(seqs):
        existe = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL)
        if existe:
            m = re.match( r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL )
            f.write(f">{titles_list[index]}\n{m.group(1)}\n\n")                    
            #print(m.group(1))       

In [ ]:
#another easier way to do this
# from Bio import SwissProt
# from Bio import ExPASy
# with ExPASy.get_sprot_raw("P35568") as handle:
#     seq_record = SeqIO.read(handle, "swiss")
#     print(seq_record.id)
#     #print(seq_record.entry_name, "\n")
#     #print(", ".join(seq_record.accessions), "\n")
#     #print(seq_record.keywords, "\n")
#     #print(seq_record.organism, "\n")
#     #print(len(seq_record.sequence), "aa", "\n")
#     print(seq_record.seq)

In [59]:
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [60]:
file= open("CDS_protein_result_blastp.txt", "r")
fields="accession,organism_name,protein_name,cc_subcellular_location,cc_function"
WEBSITE_API="https://rest.uniprot.org"
with open('uniprot_result_CDS_filtered.txt', 'w',encoding='utf-8') as f:
    for i in file:
        r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i, fields))
        print(r.text)
        f.write(r.text)
        f.write('\n')

Entry	Organism	Protein names	Subcellular location [CC]	Function [CC]
P35568	Homo sapiens (Human)	Insulin receptor substrate 1 (IRS-1)		FUNCTION: May mediate the control of various cellular processes by insulin. When phosphorylated by the insulin receptor binds specifically to various cellular proteins containing SH2 domains such as phosphatidylinositol 3-kinase p85 subunit or GRB2. Activates phosphatidylinositol 3-kinase when bound to the regulatory p85 subunit (By similarity). {ECO:0000250, ECO:0000269|PubMed:16878150}.

Entry	Organism	Protein names	Subcellular location [CC]	Function [CC]
Q28224	Chlorocebus aethiops (Green monkey) (Cercopithecus aethiops)	Insulin receptor substrate 1 (IRS-1)		FUNCTION: May mediate the control of various cellular processes by insulin. When phosphorylated by the insulin receptor binds specifically to various cellular proteins containing SH2 domains such as phosphatidylinositol 3-kinase p85 subunit or GRB2. Activates phosphatidylinositol 3-kinase when bo

Entry	Organism	Protein names	Subcellular location [CC]	Function [CC]
B3N946	Drosophila erecta (Fruit fly)	Insulin receptor substrate 1 (Protein chico)		FUNCTION: Activates phosphatidylinositol 3-kinase when bound to the regulatory p85 subunit. May mediate the control of various cellular processes by insulin-like peptides. When phosphorylated by the insulin receptor binds specifically to various cellular proteins containing SH2 domains. Involved in control of cell proliferation, cell size, and body and organ growth throughout development. Also has a role in a signaling pathway controlling the physiological response required to endure periods of low nutrient conditions. Insulin/insulin-like growth factor (IGF) signaling pathway has a role in regulating aging and is necessary in the ovary for vitellogenic maturation (By similarity). {ECO:0000250|UniProtKB:P35570, ECO:0000250|UniProtKB:Q9XTN2}.

Entry	Organism	Protein names	Subcellular location [CC]	Function [CC]
B4NZ70	Drosophila yakuba (

Entry	Organism	Protein names	Subcellular location [CC]	Function [CC]
Q13480	Homo sapiens (Human)	GRB2-associated-binding protein 1 (GRB2-associated binder 1) (Growth factor receptor bound protein 2-associated protein 1)		FUNCTION: Adapter protein that plays a role in intracellular signaling cascades triggered by activated receptor-type kinases. Plays a role in FGFR1 signaling. Probably involved in signaling by the epidermal growth factor receptor (EGFR) and the insulin receptor (INSR). Involved in the MET/HGF-signaling pathway (PubMed:29408807). {ECO:0000269|PubMed:29408807}.

Entry	Organism	Protein names	Subcellular location [CC]	Function [CC]
Q9QYY0	Mus musculus (Mouse)	GRB2-associated-binding protein 1 (GRB2-associated binder 1) (Growth factor receptor bound protein 2-associated protein 1)		FUNCTION: Adapter protein that plays a role in intracellular signaling cascades triggered by activated receptor-type kinases. Plays a role in FGFR1 signaling. Probably involved in signaling by th

## Alignment and Phylo

In [2]:
from Bio.Align.Applications import ClustalwCommandline

In [3]:
cline = ClustalwCommandline("clustalw2", infile="allOrg_CDS_prot_fromUniprot.fasta.fasta")
print(cline)

clustalw2 -infile=allOrg_CDS_prot_fromUniprot.fasta.fasta


In [ ]:
#docker 
# docker pull quay.io/biocontainers/clustalw:2.1--hc9558a2_5
# # clustalw
# Id : 1468861e975d

In [ ]:
docker run -it --mount type=bind,source="/C/Users/Karyna/Desktop/Github/Lab-Bioinf/IRS1_last/allOrg_CDS_prot_fro
mUniprot.fasta",target=/usr quay.io/biocontainers/clustalw:2.1--hc9558a2_5 bash

clustalw -infile=/input/allOrg_CDS_prot_fromUniprot.fasta


In [5]:
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align import AlignInfo
# exp = SeqIO.parse(open("allOrg_CDS_prot_new.fasta"), format="fasta") 
# for i in exp:
#     print(len(i.seq))

The length of the seqs are all different, so it's not possible to use MultipleSeqAlignment method. CLustalW Web was used instead.

In [6]:
# record_align = SeqIO.parse(open("allOrg_CDS_prot.fasta"), format="fasta")
# record1 = SeqIO.parse(open("allOrg_CDS_prot.fasta"), format="fasta")

# alin1 = MultipleSeqAlignment(record1)
# print(alin1)
# print(alin1.get_alignment_length())

In [ ]:
seqs = []
for seq in record1:
    seqs.append(seq)

alin1 = MultipleSeqAlignment(seqs)
print(alin1)
print(alin1.get_alignment_length())